In [1]:
#ADD SCHEDULER

import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
tick = pd.read_csv("all_stocks")
final = pd.DataFrame(columns=[
                            'Ticker',
                            'Price',
                            '1y Price Return',
                            '1y percentile', 
                            '6m Price Return', 
                            '6m percentile', 
                            '3m Price Return', 
                            '3m percentile',
                            '1m Price Return', 
                            '1m percentile',
                            'Weighted Hurst Exponent'
                            ])
counter= 0

#calulate the Hurst exponent of a stock
def get_hurst_exponent(time_series, max_lag):
    """Returns the Hurst Exponent of the time series"""
        
    lags = range(2, max_lag)

    # variances of the lagged differences
    tau = [np.std(np.subtract(time_series[lag:], time_series[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(lags), np.log(tau), 1)

    return reg[0]

for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        max = yf.Ticker(tick["0"][i]).history(period='max')
        d = yf.Ticker(tick["0"][i]).history(period='max')
        y = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
        m6 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
        m3 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
        m1 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=1))
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        continue
    
    max.reset_index(inplace = True)
    d.reset_index(inplace=True)
    y.reset_index(inplace=True)
    m6.reset_index(inplace=True)
    m3.reset_index(inplace=True)
    m1.reset_index(inplace=True)

    d['Ticker'] = tick['0'][i]    
    try: 
        perc_change1y = ((y.iloc[-1]['Close'] - y.iloc[0]['Close'])/y.iloc[0]["Close"])
        d['1y Price Return'] = perc_change1y
        perc_change6m = ((m6.iloc[-1]['Close'] - m6.iloc[0]['Close'])/m6.iloc[0]["Close"])
        d['6m Price Return'] = perc_change6m
        perc_change3m = ((m3.iloc[-1]['Close'] - m3.iloc[0]['Close'])/m3.iloc[0]["Close"])
        d['3m Price Return'] = perc_change3m
        perc_change1m = ((m1.iloc[-1]['Close'] - m1.iloc[0]['Close'])/m1.iloc[0]["Close"])
        d['1m Price Return'] = perc_change1m

        d['Price'] = d['Close'] 
        d['1y percentile'] = 0
        d['6m percentile'] = 0
        d['3m percentile'] = 0
        d['1m percentile'] = 0
        d['Weighted Hurst Exponent'] = 0
        d = d.iloc[-1]
    except IndexError: 
        continue

    #Change to keep only last 2 days!!! get date

    t = [20, 100, 300, 500, 1000]
    total_hurst = 0
    weight = 0.4
    for j in range(len(t)): 
        #Use Weighted average to determine hurst exp of the stock
        hurst_exp = get_hurst_exponent(max["Close"].values, t[j])
        #print(tick["0"][i] + " Hurst exponent with " + str(t[j]) +  ' lags: ' + str(hurst_exp))
        total_hurst += hurst_exp*weight
        if j >= 2: 
            weight = 0.1
        else: 
            weight = 0.2
        print(total_hurst)
    d['Weighted Hurst Exponent'] = total_hurst
    
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker',
                            'Price',
                            '1y Price Return',
                            '1y percentile', 
                            '6m Price Return', 
                            '6m percentile', 
                            '3m Price Return', 
                            '3m percentile',
                            '1m Price Return', 
                            '1m percentile',
                            'Weighted Hurst Exponent'
                            ]), 
        ignore_index = True
    ) 

final.to_csv("momentum_advanced_sheet")

#Get percentiles for each stock
cols = ['1y', '6m', '3m', '1m']
top90 = []
for c in cols: 
    for index, row in final.iterrows(): 
        percentile_change = stats.percentileofscore(final[c+ ' Price Return'], final[c + ' Price Return'].loc[index])
        final[c + ' percentile'][index] = percentile_change 
final['Shares To Buy'] = 0

#Calculate HQM score
#Get the mean of all 4 percentiles 
final['HQM score'] = 0
from statistics import mean
for index, row in final.iterrows(): 
    all_periods_p = [row['1y percentile'], row['6m percentile'], row['3m percentile'], row['1m percentile']]
    final['HQM score'].iloc[index] = mean(all_periods_p)

#Sort values to present
final.sort_values("HQM score", ascending=False, inplace = True)
final.to_csv("momentum_advanced_HQM")

Producing data for:  AAPL


<ipython-input-1-d5b62f59ed62>:47: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-1-d5b62f59ed62>:48: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-1-d5b62f59ed62>:49: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=3))
<ipython-input-1-d5b62f59ed62>:50: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime 

0.19374906382831417
0.28411129693335657
0.3829721611658028
0.4381515787942888
0.4965638509727813
Producing data for:  TSLA


<ipython-input-1-d5b62f59ed62>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(
<ipython-input-1-d5b62f59ed62>:47: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-1-d5b62f59ed62>:48: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-1-d5b62f59ed62>:49: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3

0.2098941993072032
0.3029359694688357
0.3911055984384126
0.43738136079578205
0.48139083353221335
Producing data for: 

...

0.18661956394578824
0.27967611029829276
0.3592218856937129
0.3915701610058411
0.4217914638985807
Producing data for:  BK


<ipython-input-1-d5b62f59ed62>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(
<ipython-input-1-d5b62f59ed62>:47: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  y = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=12))
<ipython-input-1-d5b62f59ed62>:48: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m6 = max.truncate(before=pd.Timestamp('today') - pd.DateOffset(months=6))
<ipython-input-1-d5b62f59ed62>:49: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  m3

0.1782343519095513
0.2631205987100476
0.35226748978620726
0.39586599793101457
0.4364838666001064
Producing data for:  BNS


<ipython-input-1-d5b62f59ed62>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final = final.append(
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub_missing[list(keys)] = _np.nan
C:\Users\Kamen\AppData\Local\Programs\Python\Python38\lib\site-packages\yfinance\utils.py:613: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

ValueError: truncate requires a sorted index